In [ ]:
# installing anvil
!pip install anvil-uplink

In [ ]:
# importing neccessary packages
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models, optimizers, regularizers
import anvil.media
import anvil.server
import io
from IPython.display import Image, display
import matplotlib.cm as cm
from keras.preprocessing.image import load_img

In [ ]:
# loads trained model from memory
same_model = keras.models.load_model("my_model")
same_model.summary()

In [ ]:
# sets data path to dataset
data_path = '/kaggle/input/data256/Data256'

# parameter constants
batch_size = 128
img_height = 256
img_width = 256

# creates training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  color_mode='grayscale',
  image_size=(img_height, img_width),
  batch_size=batch_size)

# creates testing dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
  data_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  color_mode='grayscale',
  image_size=(img_height, img_width),
  batch_size=batch_size)

# sets class names
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

# applies prefetch transformation input pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# normalizes data pixel values between 0-1 and creates data batches
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
normalized_test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
test_image_batch, test_labels_batch = next(iter(normalized_test_ds))

In [ ]:
# creates data augmentation layer
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(256,
                                  256,
                                  1)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

# creates CNN model
model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(256, 256, 1)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(512, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

# compiles CNN model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
# trains model for 100 epochs
epochs = 100
history = model.fit(
  normalized_train_ds,
  epochs = epochs,
  validation_data = normalized_test_ds 
)

In [ ]:
# evaluates model accuracy on testing dataset
model.evaluate(normalized_test_ds)

In [ ]:
# connects to anvil website server
anvil.server.connect("server_LQLZDGGDEATEV563AGBYES73-GEMOTF2RRYNYHUIC")

In [ ]:
# returns predictions of model on image
@anvil.server.callable
def predict_image(img1):
    probabilities = [0, 0, 0]
    with anvil.media.TempFile(img1) as i:
        img2 = tf.keras.utils.load_img(i)
    img_color = img2.resize((256,256)) 
    img = img_color.convert("L")
    img_array = tf.keras.utils.img_to_array(img)
    img_normalized = img_array.astype(np.float) / 255
    img_batch = tf.expand_dims(img_normalized, 0)
    predictions = same_model.predict(img_batch)
    predictions_final = tf.nn.softmax(predictions[0])
    for i in range(3):
        probabilities[i] = float(predictions_final[i])  
    prediction = class_names[np.argmax(predictions_final)]

    return probabilities, prediction

In [ ]:
# converts input image to JPEG
def img_to_media_obj(img):
  img_byte_arr = io.BytesIO()
  img.save(img_byte_arr, format='JPEG')
  img_byte_arr = img_byte_arr.getvalue()
  media_obj = anvil.BlobMedia(content_type="image/jpeg", content=img_byte_arr)
  return media_obj